In [1]:
import json
import pandas as pd
from getpass import getpass
from openai import OpenAI

token = getpass('Token Open AI')

In [2]:
client = OpenAI(api_key=token)

In [36]:
df = pd.read_csv("../data/alpe-experiments/merged/qald-9-plus-train.tsv", sep='\t')
df = df[df.prompt_type == 'ONE_SHOT']
df = df.sample(frac=.15)

In [37]:
# Convert the dataframe to a list of dictionaries
data = df[['prompt', 'gold_standard_nl']].to_dict('records')

# Create a new list to store the transformed data
transformed_data = []

# Transform the data to the required format
for record in data:
    transformed_data.append({
        "messages": [
            {"role": "user", "content": record['prompt']},
            {"role": "assistant", "content": record['gold_standard_nl']}
        ]
    })

# Save the transformed data to a jsonl file
with open('../data/alpe-experiments/merged/qald-9-plus-train.jsonl', 'w') as f:
    for item in transformed_data:
        f.write(f"{json.dumps(item)}\n")

In [38]:
file = client.files.create(
  file=open("../data/alpe-experiments/merged/qald-9-plus-train.jsonl", "rb"),
  purpose="fine-tune"
)

In [39]:
response = client.fine_tuning.jobs.create(
  training_file=file.id,
  model="gpt-3.5-turbo"
)

In [40]:
response

FineTuningJob(id='ftjob-RtVuAeFyziHwjhHQJk5qRzOf', created_at=1712325272, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-NTsJgdgG0pdYMYQRpQjQbchJ', result_files=[], status='validating_files', trained_tokens=None, training_file='file-Uemac8bBIR5iAcDFQH2GZ08b', validation_file=None, user_provided_suffix=None, seed=187745952, integrations=[])

In [3]:
client.fine_tuning.jobs.list(limit=5)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-hIXTQzk6Sf2keTOHqiEwyISo', created_at=1712325127, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::9AfWwuZ3', finished_at=1712329950, hyperparameters=Hyperparameters(n_epochs=3, batch_size=5, learning_rate_multiplier=8), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-dVlNDXTNfBv2VG0VtX1F6n4U', result_files=['file-Xynh5QZC2h80FyvScQfQblo1'], status='succeeded', trained_tokens=3377091, training_file='file-rXM0Xvkq56GIY4e6mk7wzrQR', validation_file=None, user_provided_suffix=None, seed=1241654507, integrations=[])], object='list', has_more=False)